In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
# Define cubic SE6, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
Lambda3 = {b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}
sost_SE6 = {f: -c*(5*c+e)/(c+e)}
SE6 = general_cubic.subs(Lambda3).subs(sost_SE6)

In [5]:
# Find simmetries

In [6]:
%time adm_SE6 = SE6.find_admissible_projectivities()
len(adm_SE6)

CPU times: user 466 ms, sys: 56.3 ms, total: 522 ms
Wall time: 1.26 s


48

In [7]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)
len(simm_SE6)

CPU times: user 219 ms, sys: 53.1 ms, total: 272 ms
Wall time: 385 ms


24

In [8]:
### geometric interpretation of the group of symmetries

In [9]:
## Action of the stabilizer of SE6 on the 27 lines:
Ga = perm_group_lines(SE6, simm_SE6)

## Action of the stabilizer of SE6 on the 45 tritangent planes:
Gb = perm_group_planes(SE6, simm_SE6)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
S4

Structure of the group Gb:
S4


In [10]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [11]:
len(OGa), len(OGb)

(3, 7)

In [12]:
## The orbits OGa are a partition of the set {1, 2, ..., 27}:
OGa

[[1, 3, 4, 7, 8, 9, 13, 14, 15, 18, 22, 27],
 [2, 19, 10],
 [5, 25, 26, 6, 16, 24, 20, 11, 12, 21, 23, 17]]

In [13]:
## We convert the elements of the orbits of OGa (which are numbers from 1 to 27) 
## into the corresponding lines expressed with the labels:
for orb in OGa:
    print([simbolic_line_from_index(i) for i in orb])

['E1', 'E3', 'E4', 'G1', 'G2', 'G3', 'F12', 'F13', 'F14', 'F23', 'F34', 'F56']
['E2', 'F24', 'G4']
['E5', 'F45', 'F46', 'E6', 'F15', 'F36', 'F25', 'G5', 'G6', 'F26', 'F35', 'F16']


In [14]:
## The orbits OGb are a partition of the set {1, 2, ..., 45}:
OGb

[[1, 11, 18, 37],
 [2, 12, 31, 16],
 [3, 17, 6, 34, 13, 7],
 [4, 22, 33, 35, 14, 26, 28, 20, 42, 43, 39, 25],
 [5, 27, 32, 36, 15, 21, 23, 19, 45, 40, 38, 30],
 [8],
 [9, 41, 24, 29, 44, 10]]

In [15]:
for orb in OGb:
    print([simbolic_plane_from_index(i) for i in orb])

['t1', 't11', 't18', 't37']
['t2', 't12', 't31', 't16']
['t3', 't17', 't6', 't34', 't13', 't7']
['t4', 't22', 't33', 't35', 't14', 't26', 't28', 't20', 't42', 't43', 't39', 't25']
['t5', 't27', 't32', 't36', 't15', 't21', 't23', 't19', 't45', 't40', 't38', 't30']
['t8']
['t9', 't41', 't24', 't29', 't44', 't10']


In [16]:
## The group Gb permutes the elements of the first orbit of OGb (which has 4 elements)
## in all the 4! possibilities:
O1 = OGb[0]
totPerm = []
for gg in Gb:
    totPerm.append(tuple([gg(i) for i in O1]))

In [17]:
len(Set(totPerm))

24

In [18]:
print(totPerm)

[(11, 37, 1, 18), (37, 18, 1, 11), (11, 1, 37, 18), (18, 1, 11, 37), (37, 11, 1, 18), (37, 1, 18, 11), (1, 37, 18, 11), (1, 37, 11, 18), (1, 18, 11, 37), (37, 11, 18, 1), (11, 1, 18, 37), (18, 37, 1, 11), (18, 1, 37, 11), (11, 18, 37, 1), (11, 37, 18, 1), (18, 11, 1, 37), (18, 37, 11, 1), (37, 1, 11, 18), (37, 18, 11, 1), (1, 11, 37, 18), (1, 11, 18, 37), (1, 18, 37, 11), (11, 18, 1, 37), (18, 11, 37, 1)]


In [19]:
### hence OGb is S_4.

In [20]:
### the same is true for the second orbit:
O2 = OGb[1]
totPerm = []
for gg in Gb:
    totPerm.append(tuple([gg(i) for i in O2]))

In [21]:
len(Set(totPerm))

24

In [22]:
print(totPerm)

[(12, 16, 2, 31), (31, 16, 12, 2), (12, 2, 16, 31), (2, 31, 16, 12), (12, 16, 31, 2), (31, 12, 16, 2), (31, 12, 2, 16), (12, 31, 2, 16), (2, 31, 12, 16), (16, 12, 31, 2), (2, 12, 16, 31), (31, 16, 2, 12), (31, 2, 16, 12), (16, 2, 12, 31), (16, 12, 2, 31), (2, 16, 31, 12), (16, 31, 2, 12), (12, 31, 16, 2), (16, 31, 12, 2), (12, 2, 31, 16), (2, 12, 31, 16), (31, 2, 12, 16), (2, 16, 12, 31), (16, 2, 31, 12)]


In [23]:
SE6.eckardt_points_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G1', 'F12'],
 ['E2', 'G3', 'F23'],
 ['E3', 'G4', 'F34'],
 ['E4', 'G2', 'F24'],
 ['F13', 'F24', 'F56']]

In [24]:
P1, P2, P3, P4, P5, P6 = SE6.eckardt_points

In [25]:
## study of the allignments of the Eckardt points
matrix([P1, P2, P5]).rank(), matrix([P1, P3, P6]).rank(), \
matrix([P2, P4, P6]).rank(), matrix([P3, P4, P5]).rank()

(2, 2, 2, 2)